In [20]:
%load_ext autoreload
%autoreload 2
from datetime import date

import polars as pl

from property_models.constants import DATA_DIR, POSTCODE_CSV_FILE, PropertyType, RecordType
from property_models.models import Address, HistoricalPrice, PropertyInfo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
PropertyType.parse(PropertyType.APARTMENT.SIXTIES_BRICK.value)

NotImplementedError: 

('a', 'b')

In [16]:
PropertyInfo(
    property_type=PropertyType.parse(PropertyType.APARTMENT.SIXTIES_BRICK),
    address=Address.parse("80 ROSEBERRY STREET, NORTH MELBOURNE, VIC 3032", country="australia"),
    condition=None,
    bed=10,
    car=10,
    bath=10,
    floor_count=10,
    land_size_m2=100.3,
    property_size_m2=304.4,
    date_of_construction=date(2000, 1, 1),
)

PropertyInfo(address=Address(unit_number=None, street_number=80, street_name='ROSEBERRY STREET', suburb='NORTH MELBOURNE', post_code=3032, state='VIC', country='australia'), bed=10, bath=10, car=10, property_size_m2=304.4, land_size_m2=100.3, condition=None, property_type=('apartment', 'sixties_brick'), date_of_construction=datetime.date(2000, 1, 1), floor_count=10)